Goal of this notebook:

Investigate the impact of using discretized datasets (log2, log10, and mdlp) on the classification accuracy with decision tree, random forest and XGBoost.

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display

pd.set_option('display.max_columns', None)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRFClassifier
import warnings
import classifier_utils
import configs
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])

In [3]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

# Decision Tree

In [4]:
dt = DecisionTreeClassifier(random_state=99)

In [5]:
result_dt_default = classifier_utils.ProjectsResults(dt, projects, non_features_columns)

In [6]:
report_dt_default = result_dt_default.get_report_df(include_overall=True)
# report_dt_default

In [7]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_dt_log10 = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path)
report_dt_log10 = result_dt_log10.get_report_df(include_overall=True)
# report_dt_log10

In [8]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_dt_log2 = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path)
report_dt_log2 = result_dt_log2.get_report_df(include_overall=True)
# report_dt_log2

In [9]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_dt_mdlp = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path)
report_dt_mdlp = result_dt_mdlp.get_report_df(include_overall=True)

In [10]:
df_inner_dt = pd.merge(report_dt_default, report_dt_log10, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_dt_log2 = report_dt_log2.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_log2, on='project', how='inner')
df_inner_dt_mdlp = report_dt_mdlp.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_mdlp, on='project', how='inner')

accuracy_dt = df_inner_dt.filter(regex=("project|accuracy.*")).copy()
accuracy_dt['improv._log10'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_dt['improv._log2'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_dt['improv._mdlp'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_dt

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.97200,0.9680,0.97100,0.970000,-0.004115,-0.001029,-0.002058
1,apache__directory-server,0.93100,0.9190,0.91900,0.882000,-0.012889,-0.012889,-0.052632
2,jgralab__jgralab,0.83600,0.7560,0.82700,0.797000,-0.095694,-0.010766,-0.046651
3,Unidata__thredds,0.90900,0.8990,0.90000,0.856000,-0.011001,-0.009901,-0.058306
4,CloudStack-extras__CloudStack-archive,0.76000,0.7560,0.74800,0.742000,-0.005263,-0.015789,-0.023684
5,apache__accumulo,0.83400,0.8340,0.83200,0.831000,0.000000,-0.002398,-0.003597
6,Ramblurr__Anki-Android,0.69400,0.6560,0.67500,0.680000,-0.054755,-0.027378,-0.020173
7,getrailo__railo,0.64300,0.6360,0.66400,0.650000,-0.010886,0.058824,0.019608
8,TeamDev-Ltd__OpenFaces,0.96500,0.9630,0.96400,0.964000,-0.002073,-0.001036,-0.001036
9,apache__lucene-solr,0.55900,0.5700,0.56000,0.544000,0.024943,0.002268,-0.026834


## Including NA values

In [11]:
import importlib
importlib.reload(classifier_utils)
result_dt_default_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns, replace_na=True)

In [12]:
report_dt_default_wna = result_dt_default_wna.get_report_df(include_overall=True)
# report_dt_default_wna

In [13]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_dt_log2_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path, replace_na=True)
report_dt_log2_wna = result_dt_log2_wna.get_report_df(include_overall=True)
# report_dt_log2_wna

In [14]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_dt_log10_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path, replace_na=True)
report_dt_log10_wna = result_dt_log10_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [15]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_dt_mdlp_wna = classifier_utils.ProjectsResults(dt, projects, non_features_columns,discretized_path, replace_na=True)
report_dt_mdlp_wna = result_dt_mdlp_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [16]:
df_inner_dt = pd.merge(report_dt_default_wna, report_dt_log10_wna, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_dt_log2 = report_dt_log2_wna.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_log2, on='project', how='inner')
df_inner_dt_mdlp = report_dt_mdlp_wna.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_dt = pd.merge(df_inner_dt, df_inner_dt_mdlp, on='project', how='inner')

# accuracy_dt = df_inner_dt.filter(regex=("project|observations_default|accuracy.*")).copy()
accuracy_dt = df_inner_dt.filter(regex=("project|accuracy.*")).copy()
accuracy_dt['improv._log10'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_dt['improv._log2'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_dt['improv._mdlp'] = accuracy_dt.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_dt

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.971000,0.971000,0.971000,0.969000,0.000000,0.000000,-0.002060
1,apache__directory-server,0.907000,0.890000,0.879000,0.882000,-0.018743,-0.030871,-0.027563
2,jgralab__jgralab,0.819000,0.750000,0.818000,0.797000,-0.084249,-0.001221,-0.026862
3,CloudStack-extras__CloudStack-archive,0.787000,0.758000,0.773000,0.744000,-0.036849,-0.017789,-0.054638
4,Unidata__thredds,0.896000,0.893000,0.886000,0.856000,-0.003348,-0.011161,-0.044643
5,apache__accumulo,0.848000,0.848000,0.850000,0.849000,0.000000,0.013158,0.006579
6,getrailo__railo,0.730000,0.715000,0.728000,0.718000,-0.020548,-0.002740,-0.016438
7,Ramblurr__Anki-Android,0.680000,0.676000,0.667000,0.688000,-0.005882,-0.019118,0.025000
8,TeamDev-Ltd__OpenFaces,0.955000,0.958000,0.954000,0.957000,0.066667,-0.001047,0.044444
9,apache__lucene-solr,0.540000,0.537000,0.523000,0.534000,-0.005556,-0.031481,-0.011111


The overall average accuracy did not increase using discretized datasets (log2, log10, and mdlp). The same was observed when considering the dataset with and without NA values.

# Random forest

In [17]:
rf = RandomForestClassifier(random_state=99, n_jobs=5, n_estimators=400, max_features=0.3, min_samples_leaf=1)# Decision Tree

In [18]:
result_rf_default = classifier_utils.ProjectsResults(rf, projects, non_features_columns)

In [19]:
report_rf_default = result_rf_default.get_report_df(include_overall=True)
# report_rf_default

In [20]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_rf_log10 = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path)
report_rf_log10 = result_rf_log10.get_report_df(include_overall=True)
# report_rf_log10

In [21]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_rf_log2 = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path)
report_rf_log2 = result_rf_log2.get_report_df(include_overall=True)
# report_dt_log2

In [22]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_rf_mdlp = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path)
report_rf_mdlp = result_rf_mdlp.get_report_df(include_overall=True)
# report_dt_log2

In [30]:
df_inner_rf = pd.merge(report_rf_default, report_rf_log10, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_rf_log2 = report_rf_log2.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_log2, on='project', how='inner')
df_inner_rf_mdlp = report_rf_mdlp.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_mdlp, on='project', how='inner')

accuracy_rf = df_inner_rf.filter(regex=("project|accuracy.*")).copy()
accuracy_rf['improv._log10'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_rf['improv._log2'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_rf['improv._mdlp'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_rf

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.97600,0.97300,0.97500,0.973000,-0.003074,-0.001025,-0.003074
1,apache__directory-server,0.93900,0.92200,0.93100,0.910000,-0.018104,-0.008520,-0.030884
2,jgralab__jgralab,0.86900,0.81200,0.86700,0.834000,-0.065593,-0.002301,-0.040276
3,CloudStack-extras__CloudStack-archive,0.80300,0.80700,0.80700,0.774000,0.020305,0.020305,-0.036115
4,Unidata__thredds,0.91900,0.92400,0.92400,0.866000,0.061728,0.061728,-0.057671
5,apache__accumulo,0.86400,0.86100,0.86300,0.858000,-0.003472,-0.001157,-0.006944
6,Ramblurr__Anki-Android,0.74000,0.74000,0.73400,0.713000,0.000000,-0.008108,-0.036486
7,getrailo__railo,0.70100,0.69700,0.70600,0.703000,-0.005706,0.016722,0.006689
8,apache__lucene-solr,0.64400,0.63400,0.65300,0.583000,-0.015528,0.025281,-0.094720
9,TeamDev-Ltd__OpenFaces,0.96900,0.96900,0.96900,0.972000,0.000000,0.000000,0.096774


## Including NA values

In [24]:
import importlib
importlib.reload(classifier_utils)
result_rf_default_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns, replace_na=True)

In [25]:
report_rf_default_wna = result_rf_default_wna.get_report_df(include_overall=True)
# report_dt_default_wna

In [26]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_rf_log2_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path, replace_na=True)
report_rf_log2_wna = result_rf_log2_wna.get_report_df(include_overall=True)
# report_dt_log2_wna

In [27]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_rf_log10_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path, replace_na=True)
report_rf_log10_wna = result_rf_log10_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [28]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_rf_mdlp_wna = classifier_utils.ProjectsResults(rf, projects, non_features_columns,discretized_path, replace_na=True)
report_rf_mdlp_wna = result_rf_mdlp_wna.get_report_df(include_overall=True)
# report_dt_log10_wna

In [31]:
df_inner_rf = pd.merge(report_rf_default_wna, report_rf_log10_wna, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_rf_log2 = report_rf_log2_wna.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_log2, on='project', how='inner')
df_inner_rf_mdlp = report_rf_mdlp_wna.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_rf = pd.merge(df_inner_rf, df_inner_rf_mdlp, on='project', how='inner')

# accuracy_dt = df_inner_dt.filter(regex=("project|observations_default|accuracy.*")).copy()
accuracy_rf = df_inner_rf.filter(regex=("project|accuracy.*")).copy()
accuracy_rf['improv._log10'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_rf['improv._log2'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_rf['improv._mdlp'] = accuracy_rf.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_rf

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.97700,0.975000,0.977000,0.975000,-0.002047,0.000000,-0.002047
1,apache__directory-server,0.93300,0.923000,0.925000,0.910000,-0.010718,-0.008574,-0.024652
2,jgralab__jgralab,0.86700,0.813000,0.863000,0.834000,-0.062284,-0.004614,-0.038062
3,CloudStack-extras__CloudStack-archive,0.82000,0.829000,0.824000,0.790000,0.050000,0.022222,-0.036585
4,Unidata__thredds,0.91300,0.919000,0.915000,0.877000,0.068966,0.022989,-0.039430
5,apache__accumulo,0.87900,0.876000,0.877000,0.873000,-0.003413,-0.002275,-0.006826
6,getrailo__railo,0.77400,0.761000,0.777000,0.767000,-0.016796,0.013274,-0.009044
7,Ramblurr__Anki-Android,0.76300,0.746000,0.748000,0.723000,-0.022280,-0.019659,-0.052425
8,apache__lucene-solr,0.64200,0.626000,0.646000,0.576000,-0.024922,0.011173,-0.102804
9,zkoss__zk,0.80400,0.791000,0.804000,0.772000,-0.016169,0.000000,-0.039801


The overall average accuracy did not increase using discretized datasets (log2, log10, and mdlp). The same was observed when considering the dataset with and without NA values.

# XGBoost Random Forest

In [33]:
xg = XGBRFClassifier(random_state=99, subsample=0.9, eval_metric='mlogloss', n_estimators=400, colsample_bynode=0.4)

In [34]:
result_xg_default = classifier_utils.ProjectsResults(xg, projects, non_features_columns, drop_na=False)

In [35]:
report_xg_default = result_xg_default.get_report_df(include_overall=True)
# report_rf_default

In [36]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log10'
result_xg_log10 = classifier_utils.ProjectsResults(xg, projects, non_features_columns,discretized_path, drop_na=False)
report_xg_log10 = result_xg_log10.get_report_df(include_overall=True)
# report_rf_log10

In [37]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_log2'
result_xg_log2 = classifier_utils.ProjectsResults(xg, projects, non_features_columns,discretized_path, drop_na=False)
report_xg_log2 = result_xg_log2.get_report_df(include_overall=True)
# report_dt_log2

In [38]:
import importlib
importlib.reload(classifier_utils)
discretized_path = f'{configs.PROJECTS_DATA}/discretized_mdlp'
result_xg_mdlp = classifier_utils.ProjectsResults(xg, projects, non_features_columns,discretized_path, drop_na=False)
report_xg_mdlp = result_xg_mdlp.get_report_df(include_overall=True)
# report_dt_log2

In [39]:
df_inner_xg = pd.merge(report_xg_default, report_xg_log10, on='project', how='inner', suffixes=('_default', '_log10'))
df_inner_xg_log2 = report_xg_log2.add_suffix("_log2").rename(columns={"project_log2": "project"})
df_inner_xg = pd.merge(df_inner_xg, df_inner_xg_log2, on='project', how='inner')
df_inner_xg_mdlp = report_xg_mdlp.add_suffix("_mdlp").rename(columns={"project_mdlp": "project"})
df_inner_xg = pd.merge(df_inner_xg, df_inner_xg_mdlp, on='project', how='inner')

accuracy_xg = df_inner_xg.filter(regex=("project|accuracy.*")).copy()
accuracy_xg['improv._log10'] = accuracy_xg.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log10'], x['accuracy_default']), axis=1)
accuracy_xg['improv._log2'] = accuracy_xg.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_log2'], x['accuracy_default']), axis=1)
accuracy_xg['improv._mdlp'] = accuracy_xg.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_mdlp'], x['accuracy_default']), axis=1)
accuracy_xg

,project,accuracy_default,accuracy_log10,accuracy_log2,accuracy_mdlp,improv._log10,improv._log2,improv._mdlp
0,CCI-MIT__XCoLab,0.974000,0.972000,0.972000,0.974000,-0.002053,-0.002053,0.000000
1,apache__directory-server,0.914000,0.908000,0.909000,0.896000,-0.006565,-0.005470,-0.019694
2,jgralab__jgralab,0.816000,0.719000,0.798000,0.782000,-0.118873,-0.022059,-0.041667
3,Unidata__thredds,0.904000,0.899000,0.902000,0.846000,-0.005531,-0.002212,-0.064159
4,getrailo__railo,0.764000,0.746000,0.764000,0.740000,-0.023560,0.000000,-0.031414
5,apache__accumulo,0.854000,0.840000,0.848000,0.852000,-0.016393,-0.007026,-0.002342
6,TeamDev-Ltd__OpenFaces,0.962000,0.957000,0.959000,0.959000,-0.005198,-0.003119,-0.003119
7,Ramblurr__Anki-Android,0.715000,0.711000,0.704000,0.715000,-0.005594,-0.015385,0.000000
8,CloudStack-extras__CloudStack-archive,0.708000,0.704000,0.698000,0.677000,-0.005650,-0.014124,-0.043785
9,cgjones__android-frameworks-base,0.871000,0.860000,0.862000,0.864000,-0.012629,-0.010333,-0.008037


The overall average accuracy did not increase using discretized datasets (log2, log10, and mdlp). The same was observed when considering the dataset with and without NA values.